### Truncated BPTT

When the sequences are very long (thousands of points), the network training can be very slow and the memory requirements increase. The truncated BPTT is an alternative similar to mini-batch training in Dense Networks, even though in RNN the batch parameter can also be used.  

![alt text](./Images/rnn_tbptt_2.png "Truncated BPTT")

The TBPTT can be implemented by setting up the data appropriately. Let's remember that for recurrent neural networks, data must have the format **[n_samples,n_times,n_features]**, so if you want to use Truncated BPTT you just have to split the sequences into more **n_samples** of less **n_times**. 

### **BUT**, **Is it possible that the LSTM may find dependencies between the sequences?**

No it’s not possible unless you go for the stateful LSTM.

So the use of Truncated BPTT requires to set up the **Stateful** mode.

## Example

Extracted from: http://philipperemy.github.io/keras-stateful-lstm/

Let’s also a problem of classifying sequences. The data matrix $X$ is made exclusively of zeros except in the first column where exactly half of the values are 1.

In [2]:
import numpy as np
N_train = 1000
X_train = np.zeros((N_train,20))
from numpy.random import choice
one_indexes = choice(a=N_train, size=int(N_train / 2), replace=False)
X_train[one_indexes, 0] = 1  # very long term memory.
#--------------------------------
N_test = 200
X_test = np.zeros((N_test,20))
from numpy.random import choice
one_indexes = choice(a=N_test, size=int(N_test / 2), replace=False)
X_test[one_indexes, 0] = 1  # very long term memory.
print(X_train[:10,:2])
print(X_test[:10,:2])

[[1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]]


In [3]:
print(X_train.shape)

(1000, 20)


In [82]:
def prepare_sequences(x_train, y_train, window_length, increment):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1, increment):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [84]:
#Split the sequences into two sequences of length 10
window_length = 10

x_train, y_train = prepare_sequences(X_train, X_train[:,0], window_length,window_length)
x_test, y_test = prepare_sequences(X_test, X_test[:,0], window_length,window_length)
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2000, 10, 1)
(2000, 1)
(400, 10, 1)
(400, 1)


Every sequence was split into 2 subsequences

In [89]:
print(x_train[:4,:])
print(y_train[:4,:])

[[[1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]]
[[1.]
 [1.]
 [0.]
 [0.]]


### Let's train a regular LSTM network

In [9]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

### Using the original sequences:

In [100]:
print('Building STATELESS model...')
max_len = 10
batch_size = 11
model = Sequential()
model.add(LSTM(10, input_shape=(20, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.reshape(1000,20,1), X_train[:,0].reshape(1000,1), batch_size=batch_size, epochs=5,
          validation_data=(X_test.reshape(200,20,1), X_test[:,0].reshape(200,1)), shuffle=False)
score, acc = model.evaluate(X_test.reshape(200,20,1),X_test[:,0].reshape(200,1), batch_size=batch_size, verbose=0)

Building STATELESS model...
Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.6934 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/5
1000/1000 [==============================] - 1s 980us/sample - loss: 0.6918 - acc: 0.5390 - val_loss: 0.6737 - val_acc: 0.5000
Epoch 3/5
1000/1000 [==============================] - 1s 989us/sample - loss: 0.3226 - acc: 0.9170 - val_loss: 0.1435 - val_acc: 1.0000
Epoch 4/5
1000/1000 [==============================] - 1s 983us/sample - loss: 0.0979 - acc: 1.0000 - val_loss: 0.0658 - val_acc: 1.0000
Epoch 5/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.0478 - acc: 1.0000 - val_loss: 0.0322 - val_acc: 1.0000


In [15]:
acc

1.0

### Using the splitted sequences:

In [85]:
print('Building STATELESS model...')
max_len = 10
batch_size = 2
model = Sequential()
model.add(LSTM(10, input_shape=(max_len, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=15,
          validation_data=(x_test, y_test), shuffle=False)
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

Building STATELESS model...
Train on 2000 samples, validate on 400 samples
Epoch 1/15
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.5096 - acc: 0.7430 - val_loss: 0.4801 - val_acc: 0.7500
Epoch 2/15
2000/2000 [==============================] - 6s 3ms/sample - loss: 0.4809 - acc: 0.7500 - val_loss: 0.4794 - val_acc: 0.7500
Epoch 3/15
2000/2000 [==============================] - 6s 3ms/sample - loss: 0.4803 - acc: 0.7500 - val_loss: 0.4792 - val_acc: 0.7500
Epoch 4/15
2000/2000 [==============================] - 7s 4ms/sample - loss: 0.4799 - acc: 0.7500 - val_loss: 0.4790 - val_acc: 0.7500
Epoch 5/15
2000/2000 [==============================] - 7s 3ms/sample - loss: 0.4796 - acc: 0.7500 - val_loss: 0.4788 - val_acc: 0.7500
Epoch 6/15
2000/2000 [==============================] - 6s 3ms/sample - loss: 0.4794 - acc: 0.7500 - val_loss: 0.4787 - val_acc: 0.7500
Epoch 7/15
2000/2000 [==============================] - 6s 3ms/sample - loss: 0.4793 - acc: 0.7500 - val_loss

In [86]:
acc

0.75

The sequences composed of 0s are correctly classified. The subsequences starting with 1 are correctly classified, but the sebsequences of class 1 starting with 0, are wrong classified. Those are the 25% of the sequences.

### What happened?

The long range memory required to classify the sequences correctly has been lost because of the sequences' partition. 

### STATEFUL Model

In [101]:
print('Build STATEFUL model...')
max_len = 10
n_partitions = 2
batch = 1
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch, max_len, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Build STATEFUL model...


In [102]:
print('Train...')

for epoch in range(5):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(0,x_train.shape[0],n_partitions):
        #print(i)
        for j in range(n_partitions):
            #print(j)
            tr_loss, tr_acc = model.train_on_batch(x_train[i+j,:,:].reshape(1,max_len,1), y_train[i+j,:].reshape(1,1))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()    
    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(0,x_test.shape[0],n_partitions):
        for j in range(n_partitions):
            te_loss, te_acc = model.test_on_batch(x_test[i+j,:,:].reshape(1,max_len,1), y_test[i+j,:].reshape(1,1))
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')

Train...
accuracy training = 0.9769999980926514
loss training = 0.07070142775774002
___________________________________
accuracy testing = 1.0
loss testing = 0.0010626701405271888
___________________________________
accuracy training = 1.0
loss training = 0.00050863076467067
___________________________________
accuracy testing = 1.0
loss testing = 0.0002185764751629904
___________________________________
accuracy training = 1.0
loss training = 0.00012640494969673455
___________________________________
accuracy testing = 1.0
loss testing = 6.696998025290668e-05
___________________________________
accuracy training = 1.0
loss training = 4.0605660615256056e-05
___________________________________
accuracy testing = 1.0
loss testing = 2.2503991203848273e-05
___________________________________
accuracy training = 1.0
loss training = 1.3922774996899534e-05
___________________________________
accuracy testing = 1.0
loss testing = 7.854175237298477e-06
___________________________________


The code was a bit more difficult to write because we have to manually call **model.reset_states()** at each new sequence processed. Another method to do that is to write a callback that reset the states at each sequence like this:

In [106]:
from tensorflow.keras.callbacks import Callback
n_partitions = 2
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % n_partitions == 0:
            self.model.reset_states()
        self.counter += 1
        
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch, max_len, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, callbacks=[ResetStatesCallback()], batch_size=1, shuffle=False)

Epoch 1/5
2000/2000 [==============================] - 14s 7ms/sample - loss: 0.0871 - acc: 0.9595
Epoch 2/5
2000/2000 [==============================] - 11s 6ms/sample - loss: 5.0814e-04 - acc: 1.0000
Epoch 3/5
2000/2000 [==============================] - 11s 6ms/sample - loss: 1.2809e-04 - acc: 1.0000
Epoch 4/5
2000/2000 [==============================] - 11s 6ms/sample - loss: 4.1980e-05 - acc: 1.0000
Epoch 5/5
2000/2000 [==============================] - 11s 6ms/sample - loss: 1.4765e-05 - acc: 1.0000


When the dataset for validation have a different batchsize, the best way to solve it, is to create a new model with the new batchsize and transfer to it the weights of the trained model.

***Example**: The following code does not have relation with the previous examples!

In [ ]:
# configure network
n_batch = 3
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()
# re-define the batch size
n_batch = 1
# re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
new_model.add(Dense(1))
# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)
# compile model
new_model.compile(loss='mean_squared_error', optimizer='adam')